# Sagecal 

This notebook demonstrates the SageCal algorithm, based on the paper:
Radio interferometric calibration with SAGE.

S Kazemi, S Yatawatta, S Zaroubi, P Lampropoulos, A G de Bruyn, L V E Koopmans, and J Noordam.

Monthly Notices of the Royal Astronomical Society, 2011 vol. 414 (2) pp. 1656-1666.

http://adsabs.harvard.edu/cgi-bin/nph-data_query?bibcode=2011MNRAS.414.1656K&link_type=EJOURNAL


In [ ]:
%matplotlib inline

import os
import sys
sys.path.append(os.path.join('..','..'))

results_dir = './results'
os.makedirs(results_dir, exist_ok=True)

from matplotlib import pylab

pylab.rcParams['figure.figsize'] = (12.0, 12.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs.utils import pixel_to_skycoord

from matplotlib import pyplot as plt

from arl.skycomponent.operations import find_skycomponents, find_nearest_component
from arl.calibration.solvers import solve_gaintable
from arl.calibration.operations import apply_gaintable, create_gaintable_from_blockvisibility
from arl.data.data_models import Image
from arl.data.polarisation import PolarisationFrame
from arl.data.parameters import get_parameter
from arl.visibility.base import create_blockvisibility, copy_visibility
from arl.skycomponent.operations import create_skycomponent
from arl.image.operations import show_image, export_image_to_fits, qa_image, copy_image, create_empty_image_like
from arl.visibility.iterators import vis_timeslice_iter
from arl.visibility.coalesce import convert_visibility_to_blockvisibility
from arl.util.testing_support import create_named_configuration, create_low_test_beam, \
    simulate_gaintable, create_low_test_skycomponents_from_gleam
from arl.skycomponent.operations import apply_beam_to_skycomponent
from arl.imaging import create_image_from_visibility, advise_wide_field, predict_skycomponent_visibility
from arl.imaging.imaging_context import invert_function, predict_function
from arl.pipelines.functions import ical

from arl.calibration.sagecal import sagecal_solve

import logging

log = logging.getLogger()
log.setLevel(logging.INFO)
log.addHandler(logging.StreamHandler(sys.stdout))

We make the visibility. The parameter rmax determines the distance of the furthest antenna/stations used. All over parameters are determined from this number.

In [ ]:
nfreqwin = 1
ntimes = 7
rmax = 300
npixel = 1024
cellsize = 0.001
frequency = numpy.linspace(0.8e8, 1.2e8, nfreqwin)
if nfreqwin > 1:
    channel_bandwidth = numpy.array(nfreqwin * [frequency[1] - frequency[0]])
else:
    channel_bandwidth = [0.4e8]
times = numpy.linspace(-numpy.pi / 3.0, numpy.pi / 3.0, ntimes)

phasecentre = SkyCoord(
    ra=-60.0 * u.deg, dec=-60.0 * u.deg, frame='icrs', equinox='J2000')

lowcore = create_named_configuration('LOWBD2', rmax=rmax)

block_vis = create_blockvisibility(
    lowcore,
    times,
    frequency=frequency,
    channel_bandwidth=channel_bandwidth,
    weight=1.0,
    phasecentre=phasecentre,
    polarisation_frame=PolarisationFrame("stokesI"))

print(block_vis.vis.shape)

In [ ]:
wprojection_planes=1
advice=advise_wide_field(block_vis, guard_band_image=5.0, delA=0.02, wprojection_planes=wprojection_planes)

vis_slices = advice['vis_slices']
npixel=advice['npixels2']
cellsize=advice['cellsize']

Generate the model from the GLEAM catalog, including application of the primary beam.

In [ ]:
beam = create_image_from_visibility(
    block_vis,
    npixel=npixel,
    frequency=frequency,
    nchan=nfreqwin,
    cellsize=cellsize,
    phasecentre=phasecentre)

gleam_components = create_low_test_skycomponents_from_gleam(
    flux_limit=1.5,
    phasecentre=phasecentre,
    frequency=frequency,
    polarisation_frame=PolarisationFrame('stokesI'),
    radius=npixel * cellsize)

beam = create_low_test_beam(beam)
gleam_components = apply_beam_to_skycomponent(gleam_components, beam, flux_limit=1.5)
show_image(beam, components=gleam_components, cm='Greys', title='Primary beam plus original GLEAM components')
print("Number of components %d" % len(gleam_components))

Generate the template image

In [ ]:
model = create_image_from_visibility(block_vis, npixel=npixel, frequency=[numpy.average(frequency)], nchan=1,
    channel_bandwidth=[numpy.sum(channel_bandwidth)], cellsize=cellsize, phasecentre=phasecentre)

Create the model visibilities, applying a different gain table for each.

In [ ]:
corrupted_vis = copy_visibility(block_vis)
gt = create_gaintable_from_blockvisibility(block_vis, timeslice='auto')
for sc in gleam_components:
    component_vis = copy_visibility(block_vis, zero=True)
    gt = simulate_gaintable(gt, amplitude_error=0.0, phase_error=1.0)
    component_vis = predict_skycomponent_visibility(component_vis, sc)
    component_vis = apply_gaintable(component_vis, gt)
    corrupted_vis.data['vis'][...]+=component_vis.data['vis'][...]
    
dirty, sumwt = invert_function(corrupted_vis, model, context='2d')

Show the dirty image, along with the GLEAM components

In [ ]:
show_image(dirty, components=gleam_components, cm='Greys', title='Dirty image plus original components')
qa = qa_image(dirty)
print(qa)
plt.show()

Find the components above the threshold 10 times the median-abs of the dirty image

In [ ]:
qa = qa_image(dirty)
found_components= find_skycomponents(dirty, threshold=10.0*qa.data['medianabs'])
show_image(dirty, components=found_components, cm='Greys', title='Dirty image plus found components')
plt.show()

First do an isoplanatic selfcalibration using these components

In [ ]:
predicted_vis = copy_visibility(block_vis, zero=True)
predicted_vis = predict_skycomponent_visibility(predicted_vis, found_components)
gt = solve_gaintable(corrupted_vis, predicted_vis, phase_only=True, timescale='auto', seed=None)
corrupted_vis = apply_gaintable(corrupted_vis, gt, inverse=True)
dirty, sumwt = invert_function(corrupted_vis, model, context='2d')

qa = qa_image(dirty)
found_components= find_skycomponents(dirty, threshold=20.0*qa.data['medianabs'])
show_image(dirty, components=found_components, cm='Greys', title='Iso only: Dirty image plus found components')
plt.show()

Show the components found

In [ ]:
flux_original = []
flux_error = []
separation=[]
for sc in found_components:
    found = find_nearest_component(sc.direction, gleam_components)
    flux_original.append(found.flux[0,0])
    flux_error.append(found.flux[0,0]-sc.flux[0,0])
    separation.append(sc.direction.separation(found.direction).to('rad').value)
    
plt.clf()
plt.plot(flux_original, flux_error, '.')
plt.title('iso only: Error in recovered flux vs flux')
plt.xlabel('Original flux')
plt.ylabel('Error in recovered flux (original - recovered)')
plt.show()

plt.clf()
plt.plot(flux_original, separation, '.')
plt.title('iso only:Separation vs flux')
plt.xlabel('Original flux')
plt.ylabel('Separation (rad)')
plt.show()

Define callback for monitoring

In [ ]:
from arl.calibration.operations import qa_gaintable
ncomps = len(found_components)
niter=30
flux_history = numpy.zeros([ncomps, niter])
residual_history = numpy.zeros([ncomps, niter])
phase_rms_history = numpy.zeros([ncomps, niter])

def sagecal_callback(iter, thetas):
    for i, theta in enumerate(thetas):
        flux_history[i, iter] = theta[0].flux[0,0]
        qa = qa_gaintable(theta[1])
        residual_history[i, iter]=qa.data['residual']
        phase_rms_history[i, iter]=qa.data['rms-phase']   

Run sagecal

In [ ]:
thetas, residual_vis = sagecal_solve(corrupted_vis, found_components, niter=niter, gain=0.25, tol=1e-8,
                                    callback=sagecal_callback)

Show the convergence behaviour

In [ ]:
plt.clf()
for comp in range(ncomps):
    plt.plot(flux_history[comp, :], label=str(comp))

plt.title("Flux convergence")
plt.xlabel('Iteration')
plt.ylabel('Flux of component (Jy)')
plt.legend()
plt.show()

plt.clf()
for comp in range(ncomps):
    plt.semilogy(residual_history[comp, 1:], label=str(comp))

plt.title("Residual convergence")
plt.xlabel('Iteration')
plt.ylabel('Residual of gain fit (Jy)')
plt.legend()
plt.show()

plt.clf()
for comp in range(ncomps):
    plt.plot(phase_rms_history[comp, 1:], label=str(comp))

plt.title("Phase rms convergence")
plt.xlabel('Iteration')
plt.ylabel('Phase rms of fit (rad)')
plt.legend()
plt.show()

plt.clf()
for comp in range(ncomps):
    plt.semilogy(phase_rms_history[comp, 2:], residual_history[comp, 2:], label=str(comp))

plt.title("Convergence trajectory")
plt.ylabel('Residual of gain fit (Jy)')
plt.xlabel('Phase rms of fit (rad)')
plt.legend()
plt.show()

In [ ]:
flux_original = []
flux_error = []
separation=[]
for theta in thetas:
    sc=theta[0]
    found = find_nearest_component(sc.direction, gleam_components)
    flux_original.append(found.flux[0,0])
    flux_error.append(found.flux[0,0]-sc.flux[0,0])
    separation.append(sc.direction.separation(found.direction).to('rad').value)
    
plt.clf()
plt.plot(flux_original, flux_error, '.')
plt.title('sagecal: Error in recovered flux vs flux')
plt.xlabel('Original flux')
plt.ylabel('Error in recovered flux (original - recovered)')
plt.show()

plt.clf()
plt.plot(flux_original, separation, '.')
plt.title('sagecal: Separation vs flux')
plt.xlabel('Original flux')
plt.ylabel('Separation (rad)')
plt.show()

Find the componenets in the residual data

In [ ]:
show_image(dirty, components=found_components, cm='Greys', title='Dirty')
print(qa_image(dirty))
residual, sumwt = invert_function(residual_vis, model, context='2d')
show_image(residual, components=gleam_components, cm='Greys', title='Residual from Sagecal')
print(qa_image(residual))
plt.show()

In [ ]:
psf, _ = invert_function(residual_vis, model, dopsf=True, context='2d')
from arl.image.deconvolution import restore_cube
from arl.skycomponent.operations import insert_skycomponent
component_image = copy_image(model)
component_image.data[...] = 0.0
component_image= insert_skycomponent(component_image, found_components)
restored = restore_cube(component_image, psf, residual)
print(qa_image(restored, context='Restored image'))
show_image(restored, components=found_components, cm='Greys', title='Restored image using Sagecal')
plt.show()